In [1]:
import os
import sys
import ipympl

print(os.getcwd())
sys.path.insert(0, os.path.join(os.getcwd(), '../'))

/home/haruyaishikawa/dev/RecordWithRS/python/notebook


In [2]:
import numpy as np
from datetime import datetime as dt
from utils import DataManagement

In [32]:
# setup which datetime to get
dm = DataManagement()
after = dt(2018, 9, 9, 13, 7, 0)
before = dt(2018, 9, 9, 13, 8, 0)
datetimes = dm.get_datetimes_in(after, before)

assert len(datetimes) == 1
datetime = datetimes[0]

In [33]:
data_path = dm.get_save_directory(datetime)
data_path = os.path.join(data_path, 'pose')

In [34]:
files = dm.natural_sort(os.listdir(data_path))  # sorted files

## Test for one pose

In [35]:
fn = files[0]
csv_path = os.path.join(data_path, fn)

In [36]:
sys.path.insert(0, '../openpose')  # append path of openpose
from entity import params, JointType

In [37]:
# JointType
for i, joint in enumerate(JointType):
    print(i, joint.name)

0 Nose
1 Neck
2 RightShoulder
3 RightElbow
4 RightHand
5 LeftShoulder
6 LeftElbow
7 LeftHand
8 RightWaist
9 RightKnee
10 RightFoot
11 LeftWaist
12 LeftKnee
13 LeftFoot
14 RightEye
15 LeftEye
16 RightEar
17 LeftEar


In [38]:
print(JointType.LeftHand.value)

7


In [39]:
pc = np.loadtxt(csv_path, delimiter=',')

In [40]:
# Translation matrix
P_matrix_filename = os.path.join('../static_data', 'T.csv')
P = np.loadtxt(P_matrix_filename, delimiter=',')

def convert2world(P, coord):
    '''
    Convert from Camera coordniate to World coordinate (according to P)
    '''
    _coord = np.concatenate([np.asarray(coord), [1.000]])
    _P = np.array(P)
    #FIXME: Remove this when P is fixed
    rotate = np.array([[1,0,0,0], [0,-1,0,0],[0,0,-1,0],[0,0,0,1]])
    n = rotate.dot(_coord)
    return _P.dot(n)[:3]

In [41]:
rw_pc = np.zeros((18, 3))
for i, p in enumerate(pc):
    if (p == [0,0,0]).all():
        rw_pc[i] = np.asarray([np.nan, np.nan, np.nan])
    else:
        rw_pc[i] = convert2world(P, p)

In [42]:
# print(pc)
# print(rw_pc)

In [43]:
roll_pc = np.rollaxis(rw_pc, 1)

In [44]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg

In [45]:
rgb_path = dm.get_rgb_path(datetime)
rgb_image_path = os.path.join(rgb_path, '0.png')
print(rgb_image_path)
img = mpimg.imread(rgb_image_path)

/mnt/extHDD/raw_data/20180909_1307/rgb/0.png


In [55]:
%matplotlib widget

fig = plt.figure()
ax1 = fig.add_subplot(221, projection='3d')
ax1.scatter(roll_pc[0], roll_pc[1], roll_pc[2])


ax2 = fig.add_subplot(222)
ax2.imshow(img)

plt.show()

FigureCanvasNbAgg()

In [47]:
plt.savefig("compare.png")

### getting right hand location for all 1 minutes

In [48]:
del joint

joint_val = JointType.Nose.value

print("file count: ", len(files))
joint = np.zeros((len(files), 3))
for i, f in enumerate(files):
    csv_path = os.path.join(data_path, f)
    points = np.loadtxt(csv_path, delimiter=',')
    point = points[joint_val]
    if (point == [0,0,0]).all():
        joint[i] = np.asarray([np.nan, np.nan, np.nan])
    else:
        joint[i] = convert2world(P, point)

file count:  720


In [49]:
joint.shape
roll_joint = np.rollaxis(joint, 1)
roll_joint.shape

(3, 720)

In [50]:
%matplotlib widget

plt.subplot(2, 2, 1)
plt.plot(roll_joint[0], roll_joint[1])
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.suptitle('Movement of object', fontsize=16)

plt.subplot(2, 2, 2)
plt.scatter(roll_joint[0], roll_joint[1])
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()

FigureCanvasNbAgg()

In [51]:
xedges = [i for i in range(-2000, 2000, 50)]
yedges = [i for i in range(-2000, 2000, 50)]
zedges = [i for i in range(-1500, 1500, 50)]

x = roll_joint[0][~np.isnan(roll_joint[0])]
y = roll_joint[1][~np.isnan(roll_joint[1])]
z = roll_joint[2][~np.isnan(roll_joint[2])]

In [52]:
H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges))
H = H.T  # let each row list bins with common y range

In [53]:
%matplotlib widget

fig = plt.figure()

# imshow:

# ax = fig.add_subplot(131, title='imshow')
# plt.imshow(H, interpolation='nearest', origin='low',
#            extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])


# pcolor mesh:

ax = fig.add_subplot(111, title='pcolormesh',
                     aspect='equal')
X, Y = np.meshgrid(xedges, yedges)
ax.pcolormesh(X, Y, H)

# non uniform image:

# ax = fig.add_subplot(111, title='NonUniformImage',
#                      aspect='equal', xlim=xedges[[0, -1]], ylim=yedges[[0, -1]])
# im = mpl.image.NonUniformImage(ax, interpolation='bilinear')
# xcenters = (xedges[:-1] + xedges[1:]) / 2
# ycenters = (yedges[:-1] + yedges[1:]) / 2
# im.set_data(xcenters, ycenters, H)
# ax.images.append(im)
plt.show()

FigureCanvasNbAgg()

In [54]:
plt.savefig('LeftShoulder_track.png')